In [1]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest main of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 4.9 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-_7x2okwc/farm-haystack_0445b22842c44cf8a223f710669f715c
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-_7x2okwc/farm-haystack_0445b22842c44cf8a223f710669f715c
  Resolved https://github.com/deepset-ai/haystack.git to commit 820742cac7c6d8426bb9d465106193cf79944686
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4

## Logging

We configure how logging messages should be displayed and which log level should be used before importing Haystack.
Example log message:
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/218_Olenna_Tyrell.txt
Default log level in basicConfig is WARNING so the explicit parameter is not necessary but can be changed easily:

In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [3]:
# Imports needed to run this notebook

from pprint import pprint
from tqdm import tqdm
from haystack.nodes import QuestionGenerator, BM25Retriever, FARMReader
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import (
    QuestionGenerationPipeline,
    RetrieverQuestionGenerationPipeline,
    QuestionAnswerGenerationPipeline,
)
from haystack.utils import launch_es, print_questions

Let's start an Elasticsearch instance with one of the options below:

In [4]:
# Option 1: Start Elasticsearch service via Docker
launch_es()

In [5]:
# Option 2: In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

Let's initialize some core components

In [7]:
import pandas as pd 
df=pd.read_csv("/content/sample_data/df_final_T5.csv")

In [8]:
parag = []
for x in df['Pargraphs']:
  parag.append(x)

In [9]:
docs = [
    
]
for x in range(len(parag)):
  docs.append({"content" : parag[x]})

# Initialize document store and write in the documents
document_store = ElasticsearchDocumentStore()
document_store.write_documents(docs)

# Initialize Question Generator
question_generator = QuestionGenerator()

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Using sep_token, but it is not set yet.


## Question Generation Pipeline

The most basic version of a question generator pipeline takes a document as input and outputs generated questions
which the the document can answer.

In [10]:
question_generation_pipeline = QuestionGenerationPipeline(question_generator)
for idx, document in enumerate(document_store):

    print(f"\n * Generating questions for document {idx}: {document.content[:100]}...\n")
    result = question_generation_pipeline.run(documents=[document])
    print_questions(result)


 * Generating questions for document 0: historians divide ancient egyptian dynasties into the old kingdom, the middle kingdom, and the new k...


Generated questions:
 - Who divides ancient egyptian dynasties into the old kingdom, middle kingdom, and new kingdom?

 * Generating questions for document 1: about , years ago, they noticed that a star now called sirius appeared shortly before the nile began...


Generated questions:
 - What star appeared before the nile began to flood?
 - When did they notice a star called sirius?

 * Generating questions for document 2: mourners filled the tomb with objects ranging from food to furniture that the mummified pharaoh woul...


Generated questions:
 - What did mourners fill the tomb with?
 - What would the mummified pharaoh need?

 * Generating questions for document 3: a city called giza was built for the pyramid workers and the people who fed, clothed, and housed the...


Generated questions:
 - What was the name of the city built for the p

## Retriever Question Generation Pipeline

This pipeline takes a query as input. It retrieves relevant documents and then generates questions based on these.

In [11]:
retriever = BM25Retriever(document_store=document_store)
rqg_pipeline = RetrieverQuestionGenerationPipeline(retriever, question_generator)

print(f"\n * Generating questions for documents matching the query 'Arya Stark'\n")
result = rqg_pipeline.run(query="Arya Stark")
print_questions(result)


 * Generating questions for documents matching the query 'Arya Stark'


Generated questions:


## Question Answer Generation Pipeline

This pipeline takes a document as input, generates questions on it, and attempts to answer these questions using
a Reader model

In [12]:
reader = FARMReader("deepset/roberta-base-squad2")
qag_pipeline = QuestionAnswerGenerationPipeline(question_generator, reader)
for idx, document in enumerate(tqdm(document_store)):

    print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")
    result = qag_pipeline.run(documents=[document])
    print_questions(result)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.infer:Got ya 2 parallel workers to do inference ...
INFO:haystack.modeling.infer: 0     0  
INFO:haystack.modeling.infer:/w\   /w\ 
INFO:haystack.modeling.infer:/'\   / \ 
0it [00:00, ?it/s]


 * Generating questions and answers for document 0: historians divide ancient egyptian dynasties into the old kingdom, the middle kingdom, and the new k...



...


Generated pairs:
 - Q: What did egyptians mine?
      A: precious stones

 * Generating questions and answers for document 33: priests cleaned the sacred statues in temples, changed their clothes, and even fed them meals....




Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 25.95 Batches/s]
34it [00:20,  1.76it/s]


Generated pairs:
 - Q: What did priests clean the sacred statues in temples?
      A: changed their clothes, and even fed them meals
 - Q: Priests changed what?
      A: clothes

 * Generating questions and answers for document 34: egyptians believed that if a tomb was robbed, the person buried there could not have a happy afterli...




Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 41.11 Batches/s]
35it [00:21,  1.63it/s]


Generated pairs:
 - Q: Who believed that if a tomb was robbed, the person buried there could not have a happy afterlife?
      A: egyptians


In [14]:
result

{'queries': ['Wer hat Python entwickelt?',
  'Wann wurde Python zum ersten Mal veröffentlicht?',
  'Worauf konzentriert sich Pythons Designphilosophie?'],
 'documents': [[<Document: {'content': 'Python ist eine interpretierte hochsprachige Programmiersprache für allgemeine Zwecke. Sie wurde von Guido van Rossum entwickelt und 1991 erstmals veröffentlicht. Pythons Designphilosophie konzentriert sich auf die Lesbarkeit des Codes und die Verwendung von viel Leerraum (Whitespace).', 'content_type': 'text', 'score': 0.5312093733737563, 'meta': {}, 'embedding': None, 'id': '39f6dec7c3c2023157a438d1d1e702b9'}>],
  [<Document: {'content': 'Python ist eine interpretierte hochsprachige Programmiersprache für allgemeine Zwecke. Sie wurde von Guido van Rossum entwickelt und 1991 erstmals veröffentlicht. Pythons Designphilosophie konzentriert sich auf die Lesbarkeit des Codes und die Verwendung von viel Leerraum (Whitespace).', 'content_type': 'text', 'score': 0.5312093733737563, 'meta': {}, 'embed

## Translated Question Answer Generation Pipeline
Trained models for Question Answer Generation are not available in many languages other than English. Haystack
provides a workaround for that issue by machine-translating a pipeline's inputs and outputs with the
TranslationWrapperPipeline. The following example generates German questions and answers on a German text
document - by using an English model for Question Answer Generation.

In [13]:
# Fill the document store with a German document.
text1 = "Python ist eine interpretierte Hochsprachenprogrammiersprache für allgemeine Zwecke. Sie wurde von Guido van Rossum entwickelt und 1991 erstmals veröffentlicht. Die Design-Philosophie von Python legt den Schwerpunkt auf die Lesbarkeit des Codes und die Verwendung von viel Leerraum (Whitespace)."
docs = [{"content": text1}]
document_store.delete_documents()
document_store.write_documents(docs)

# Load machine translation models
from haystack.nodes import TransformersTranslator

in_translator = TransformersTranslator(model_name_or_path="Helsinki-NLP/opus-mt-de-en")
out_translator = TransformersTranslator(model_name_or_path="Helsinki-NLP/opus-mt-en-de")

# Wrap the previously defined QuestionAnswerGenerationPipeline
from haystack.pipelines import TranslationWrapperPipeline

pipeline_with_translation = TranslationWrapperPipeline(
    input_translator=in_translator, output_translator=out_translator, pipeline=qag_pipeline
)

for idx, document in enumerate(tqdm(document_store)):
    print(f"\n * Generating questions and answers for document {idx}: {document.content[:100]}...\n")
    result = pipeline_with_translation.run(documents=[document])
    print_questions(result)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,



 * Generating questions and answers for document 0: Python ist eine interpretierte Hochsprachenprogrammiersprache für allgemeine Zwecke. Sie wurde von G...



...

Translating:   0%|          | 0/3 [00:00<?, ?it/s]

Translating:   0%|          | 0/3 [00:00<?, ?it/s]

Translating:   0%|          | 0/3 [00:00<?, ?it/s]

1it [00:03,  3.09s/it]


Generated pairs:
 - Q: Wer hat Python entwickelt?
      A: Guido van Rossum
 - Q: Wann wurde Python zum ersten Mal veröffentlicht?
      A: 1991
 - Q: Worauf konzentriert sich Pythons Designphilosophie?
      A: die Lesbarkeit des Codes


In [15]:
result

{'queries': ['Wer hat Python entwickelt?',
  'Wann wurde Python zum ersten Mal veröffentlicht?',
  'Worauf konzentriert sich Pythons Designphilosophie?'],
 'documents': [[<Document: {'content': 'Python ist eine interpretierte hochsprachige Programmiersprache für allgemeine Zwecke. Sie wurde von Guido van Rossum entwickelt und 1991 erstmals veröffentlicht. Pythons Designphilosophie konzentriert sich auf die Lesbarkeit des Codes und die Verwendung von viel Leerraum (Whitespace).', 'content_type': 'text', 'score': 0.5312093733737563, 'meta': {}, 'embedding': None, 'id': '39f6dec7c3c2023157a438d1d1e702b9'}>],
  [<Document: {'content': 'Python ist eine interpretierte hochsprachige Programmiersprache für allgemeine Zwecke. Sie wurde von Guido van Rossum entwickelt und 1991 erstmals veröffentlicht. Pythons Designphilosophie konzentriert sich auf die Lesbarkeit des Codes und die Verwendung von viel Leerraum (Whitespace).', 'content_type': 'text', 'score': 0.5312093733737563, 'meta': {}, 'embed